In [1]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV

### Q1. Downloading the data

In [3]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [6]:
df.shape

(2964624, 19)

### Q2. Computing duration

In [8]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [13]:
std_duration = df['duration'].std()
std_duration

34.851053592212814

In [9]:
df_yellow = df[(df.duration >= 1) & (df.duration <= 60)]

In [10]:
df_yellow.shape

(2898906, 20)

### Q3. Dropping outliers

In [16]:
original_count = df.shape[0]
remaining_count = df_yellow.shape[0]

In [17]:
fraction_remaining = remaining_count / original_count
fraction_remaining

0.9778326020432945

### Q4. One-hot encoding

In [18]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_yellow[categorical] = df_yellow[categorical].astype(str)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13076\684723450.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yellow[categorical] = df_yellow[categorical].astype(str)


In [20]:
train_dicts = df_yellow[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [22]:
X_train.shape

(2898906, 519)

### Q5. Training a model

In [ ]:
target = 'duration'
y_train = df_yellow[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

### Q6. Evaluating the model

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [13]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet')

In [14]:
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet')

In [15]:
len(df_train), len(df_val)

(54373, 51497)

In [16]:
# Convert the location IDs to strings and concatenate
df_train['PU_DO'] = df_train['PULocationID'].astype(str) + '_' + df_train['DOLocationID'].astype(str)
df_val['PU_DO'] = df_val['PULocationID'].astype(str) + '_' + df_val['DOLocationID'].astype(str)

In [17]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\ASUS\Documents\GitHub\mlops-zoomcamp\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


5.994995068073335

In [23]:
ridge_params = {
    'alpha': [0.01, 0.1, 1.0]
}

In [26]:
# Ridge Regression
ridge = Ridge()
ridge_cv = GridSearchCV(ridge, ridge_params, scoring='neg_mean_squared_error', cv=5)
ridge_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='neg_mean_squared_error')

In [27]:
# Ridge Regression Evaluation
ridge_best = ridge_cv.best_estimator_
y_pred_ridge = ridge_best.predict(X_val)
ridge_mse = mean_squared_error(y_val, y_pred_ridge)

In [28]:

print("Best Ridge Alpha:", ridge_cv.best_params_['alpha'])
print("Ridge Validation MSE:", ridge_mse)

Best Ridge Alpha: 1.0
Ridge Validation MSE: 35.64290473399483


In [29]:
lasso_params = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
}

In [30]:
# Lasso Regression
lasso = Lasso()
lasso_cv = GridSearchCV(lasso, lasso_params, scoring='neg_mean_squared_error', cv=5)
lasso_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='neg_mean_squared_error')

In [31]:
# Lasso Regression Evaluation
lasso_best = lasso_cv.best_estimator_
y_pred_lasso = lasso_best.predict(X_val)
lasso_mse = mean_squared_error(y_val, y_pred_lasso)

In [32]:
print("Best Lasso Alpha:", lasso_cv.best_params_['alpha'])
print("Lasso Validation MSE:", lasso_mse)

Best Lasso Alpha: 100.0
Lasso Validation MSE: 83.00081112831762
